# Table of Contents
 <p><div class="lev1 toc-item"><a href="#setup" data-toc-modified-id="setup-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>setup</a></div><div class="lev1 toc-item"><a href="#isIsomorphic-code" data-toc-modified-id="isIsomorphic-code-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>isIsomorphic code</a></div><div class="lev1 toc-item"><a href="#compare-time-of-original-algorithm" data-toc-modified-id="compare-time-of-original-algorithm-3"><span class="toc-item-num">3&nbsp;&nbsp;</span>compare time of original algorithm</a></div><div class="lev1 toc-item"><a href="#compare-time-of-new-algorithm" data-toc-modified-id="compare-time-of-new-algorithm-4"><span class="toc-item-num">4&nbsp;&nbsp;</span>compare time of new algorithm</a></div>

This notebook compares the efficiency of two methods to compare reaction isomorphism, to see the time benefits of it. Two similar H_abstraction reactions are created and compared to look at time efficiency.

# setup

In [1]:
from rmgpy.species import Species
from rmgpy.reaction import Reaction
from cProfile import run

In [2]:
r1 = Species().fromSMILES('C=CC=CC=CC=CC=C[CH2]')
r2 = Species().fromSMILES('C=CC=CC=CC=C[CH2]')
p1 = Species().fromSMILES('C=CC=CC=CC=CC')
p2a = Species().fromSMILES('C=CC=CC=CC=CC=C=C')
p2b = Species().fromSMILES('C=C=C=CC=CC=CC=CC')
species = [r1,r2,p1,p2a,p2b]
for s in species: s.generateResonanceIsomers()

In [3]:
rxn1 = Reaction(reactants=[r1,r2], products = [p1, p2a])
rxn2 = Reaction(reactants=[r1,r2], products = [p1, p2b])
rxn1copy = rxn1.copy()
rxn2swapped_reactants = Reaction(reactants=[r2,r1], products = [p1, p2b])


# isIsomorphic code

In [4]:
def isIsomorphic(rxn1, other, eitherDirection=True, checkIdentical = False):
        """
        Return ``True`` if this reaction is the same as the `other` reaction,
        or ``False`` if they are different. The comparison involves comparing
        isomorphism of reactants and products, and doesn't use any kinetic
        information.

        If `eitherDirection=False` then the directions must match.
        """

        # Compare reactants to reactants
        forwardReactantsMatch = _isomorphicSpeciesList(rxn1.reactants, 
                                    other.reactants,checkIdentical = checkIdentical)
        
        # Compare products to products
        forwardProductsMatch = _isomorphicSpeciesList(rxn1.products, 
                                    other.products,checkIdentical = checkIdentical)

        # Return now, if we can
        if (forwardReactantsMatch and forwardProductsMatch):
            return True
        if not eitherDirection:
            return False
        
        # Compare reactants to products
        reverseReactantsMatch = _isomorphicSpeciesList(rxn1.reactants, 
                                    other.products,checkIdentical = checkIdentical)

        # Compare products to reactants
        reverseProductsMatch = _isomorphicSpeciesList(rxn1.products, 
                                    other.reactants,checkIdentical = checkIdentical)

        # should have already returned if it matches forwards, or we're not allowed to match backwards
        return  (reverseReactantsMatch and reverseProductsMatch)

# compare time of original algorithm

In [5]:
def _isomorphicSpeciesList(list1, list2, checkIdentical=False):
    """
    This method compares whether lists of species or molecules are isomorphic
    or identical. It is used for the 'isIsomorphic' method of Reaction class.
    It likely can be useful elswehere as well:
        
        list1 - list of species/molecule objects of reaction1
        list2 - list of species/molecule objects of reaction2
        checkIdentical - if true, uses the 'isIdentical' comparison
                         if false, uses the 'isIsomorphic' comparison
                         
    Returns True if the lists are isomorphic/identical & false otherwise
    """

    def comparison_method(other1, other2, checkIdentical=checkIdentical):
        if checkIdentical:
            return other1.isIdentical(other2)
        else:
            return other1.isIsomorphic(other2)

    if len(list1) == len(list2) == 1:
        if comparison_method(list1[0], list2[0]):
            return True
    elif len(list1) == len(list2) == 2:
        if comparison_method(list1[0], list2[0]) \
                    and comparison_method(list1[1], list2[1]):
            return True
        elif comparison_method(list1[0], list2[1]) \
                    and comparison_method(list1[1], list2[0]):
            return True
    elif len(list1) == len(list2) == 3:
        if (    comparison_method(list1[0], list2[0]) and
                comparison_method(list1[1], list2[1]) and
                comparison_method(list1[2], list2[2]) ):
            return True
        elif (  comparison_method(list1[0], list2[0]) and
                comparison_method(list1[1], list2[2]) and
                comparison_method(list1[2], list2[1]) ):
            return True
        elif (  comparison_method(list1[0], list2[1]) and
                comparison_method(list1[1], list2[0]) and
                comparison_method(list1[2], list2[2]) ):
            return True
        elif (  comparison_method(list1[0], list2[2]) and
                comparison_method(list1[1], list2[0]) and
                comparison_method(list1[2], list2[1]) ):
            return True
        elif (  comparison_method(list1[0], list2[1]) and
                comparison_method(list1[1], list2[2]) and
                comparison_method(list1[2], list2[0]) ):
            return True
        elif (  comparison_method(list1[0], list2[2]) and
                comparison_method(list1[1], list2[1]) and
                comparison_method(list1[2], list2[0]) ):
            return True
    elif len(list1) == len(list2):
        raise NotImplementedError("Can't check isomorphism of lists with {0} species/molecules".format(len(list1)))
    # nothing found
    return False

In [6]:
run('for i in range(10000): isIsomorphic(rxn1,rxn2)',)

         300003 function calls in 0.795 seconds

   Ordered by: standard name

   ncalls  tottime  percall  cumtime  percall filename:lineno(function)
    10000    0.017    0.000    0.791    0.000 <ipython-input-4-95d0327e9f94>:1(isIsomorphic)
    40000    0.059    0.000    0.773    0.000 <ipython-input-5-d3da26f90ae7>:1(_isomorphicSpeciesList)
    90000    0.706    0.000    0.706    0.000 <ipython-input-5-d3da26f90ae7>:15(comparison_method)
        1    0.005    0.005    0.795    0.795 <string>:1(<module>)
   160000    0.008    0.000    0.008    0.000 {len}
        1    0.000    0.000    0.000    0.000 {method 'disable' of '_lsprof.Profiler' objects}
        1    0.000    0.000    0.000    0.000 {range}




In [7]:
run('for i in range(10000): isIsomorphic(rxn1,rxn2swapped_reactants)')

         310003 function calls in 0.850 seconds

   Ordered by: standard name

   ncalls  tottime  percall  cumtime  percall filename:lineno(function)
    10000    0.017    0.000    0.845    0.000 <ipython-input-4-95d0327e9f94>:1(isIsomorphic)
    40000    0.061    0.000    0.828    0.000 <ipython-input-5-d3da26f90ae7>:1(_isomorphicSpeciesList)
   100000    0.759    0.000    0.759    0.000 <ipython-input-5-d3da26f90ae7>:15(comparison_method)
        1    0.005    0.005    0.850    0.850 <string>:1(<module>)
   160000    0.008    0.000    0.008    0.000 {len}
        1    0.000    0.000    0.000    0.000 {method 'disable' of '_lsprof.Profiler' objects}
        1    0.000    0.000    0.000    0.000 {range}




In [8]:
run('for i in range(10000): isIsomorphic(rxn1,rxn1copy)')

         150003 function calls in 0.969 seconds

   Ordered by: standard name

   ncalls  tottime  percall  cumtime  percall filename:lineno(function)
    10000    0.012    0.000    0.963    0.000 <ipython-input-4-95d0327e9f94>:1(isIsomorphic)
    20000    0.033    0.000    0.951    0.000 <ipython-input-5-d3da26f90ae7>:1(_isomorphicSpeciesList)
    40000    0.914    0.000    0.914    0.000 <ipython-input-5-d3da26f90ae7>:15(comparison_method)
        1    0.006    0.006    0.969    0.969 <string>:1(<module>)
    80000    0.004    0.000    0.004    0.000 {len}
        1    0.000    0.000    0.000    0.000 {method 'disable' of '_lsprof.Profiler' objects}
        1    0.000    0.000    0.000    0.000 {range}




# compare time of new algorithm

In [9]:
def _isomorphicSpeciesList(list1, list2, checkIdentical=False):
    """
    This method compares whether lists of species or molecules are isomorphic
    or identical. It is used for the 'isIsomorphic' method of Reaction class.
    It likely can be useful elswehere as well:
        
        list1 - list of species/molecule objects of reaction1
        list2 - list of species/molecule objects of reaction2
        checkIdentical - if true, uses the 'isIdentical' comparison
                         if false, uses the 'isIsomorphic' comparison
                         
    Returns True if the lists are isomorphic/identical & false otherwise
    """

    def comparison_method(other1, other2, checkIdentical=checkIdentical):
        if checkIdentical:
            return other1.isIdentical(other2)
        else:
            return other1.isIsomorphic(other2)

    list2Temp = list2[:]
    for entry in list1:
       for entry2 in list2Temp:
           if comparison_method(entry,entry2):
               list2Temp.remove(entry2)
               break
       else: #loop doesn't break, something is missing
             return False
    #all loops completed
    return True

In [10]:
run('for i in range(10000): isIsomorphic(rxn1,rxn2)',)

         160003 function calls in 0.769 seconds

   Ordered by: standard name

   ncalls  tottime  percall  cumtime  percall filename:lineno(function)
    10000    0.019    0.000    0.765    0.000 <ipython-input-4-95d0327e9f94>:1(isIsomorphic)
    40000    0.048    0.000    0.746    0.000 <ipython-input-9-8237654b1758>:1(_isomorphicSpeciesList)
    80000    0.695    0.000    0.695    0.000 <ipython-input-9-8237654b1758>:15(comparison_method)
        1    0.004    0.004    0.769    0.769 <string>:1(<module>)
        1    0.000    0.000    0.000    0.000 {method 'disable' of '_lsprof.Profiler' objects}
    30000    0.003    0.000    0.003    0.000 {method 'remove' of 'list' objects}
        1    0.000    0.000    0.000    0.000 {range}




In [11]:
run('for i in range(10000): isIsomorphic(rxn1,rxn2swapped_reactants)')

         170003 function calls in 0.832 seconds

   Ordered by: standard name

   ncalls  tottime  percall  cumtime  percall filename:lineno(function)
    10000    0.019    0.000    0.827    0.000 <ipython-input-4-95d0327e9f94>:1(isIsomorphic)
    40000    0.052    0.000    0.809    0.000 <ipython-input-9-8237654b1758>:1(_isomorphicSpeciesList)
    90000    0.753    0.000    0.753    0.000 <ipython-input-9-8237654b1758>:15(comparison_method)
        1    0.005    0.005    0.832    0.832 <string>:1(<module>)
        1    0.000    0.000    0.000    0.000 {method 'disable' of '_lsprof.Profiler' objects}
    30000    0.004    0.000    0.004    0.000 {method 'remove' of 'list' objects}
        1    0.000    0.000    0.000    0.000 {range}




In [12]:
run('for i in range(10000): isIsomorphic(rxn1,rxn1copy)')

         110003 function calls in 0.949 seconds

   Ordered by: standard name

   ncalls  tottime  percall  cumtime  percall filename:lineno(function)
    10000    0.012    0.000    0.944    0.000 <ipython-input-4-95d0327e9f94>:1(isIsomorphic)
    20000    0.037    0.000    0.932    0.000 <ipython-input-9-8237654b1758>:1(_isomorphicSpeciesList)
    40000    0.892    0.000    0.892    0.000 <ipython-input-9-8237654b1758>:15(comparison_method)
        1    0.005    0.005    0.949    0.949 <string>:1(<module>)
        1    0.000    0.000    0.000    0.000 {method 'disable' of '_lsprof.Profiler' objects}
    40000    0.004    0.000    0.004    0.000 {method 'remove' of 'list' objects}
        1    0.000    0.000    0.000    0.000 {range}


